In [28]:
!pip3 install lightgbm

In [29]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/AgriSim/fertilizer.csv")
df.head()

,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,26,52,38,Sandy,Maize,37,0,0,Urea
1,29,52,45,Loamy,Sugarcane,12,0,36,DAP
2,34,65,62,Black,Cotton,7,9,30,14-35-14
3,32,62,34,Red,Tobacco,22,0,20,28-28
4,28,54,46,Clayey,Paddy,35,0,0,Urea


In [31]:
# X = df.drop("Fertilizer Name", axis=1)
# y = df["Fertilizer Name"]
df = df.rename({'Fertilizer Name': 'Fertilizer','Crop Type': 'Crop_Type','Soil Type': 'Soil_Type'}, axis=1)


In [32]:
# One hot encoding
categorical_features=[feature for feature in df.columns if df[feature].dtype=='O']
categorical_features.remove('Fertilizer')
new_encoded_columns = pd.get_dummies(df[categorical_features])
df = pd.concat([df,new_encoded_columns],axis="columns")
df = df.drop(categorical_features,axis="columns")
df.head()

,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous,Fertilizer,Soil_Type_Black,Soil_Type_Clayey,Soil_Type_Loamy,...,Crop_Type_Cotton,Crop_Type_Ground Nuts,Crop_Type_Maize,Crop_Type_Millets,Crop_Type_Oil seeds,Crop_Type_Paddy,Crop_Type_Pulses,Crop_Type_Sugarcane,Crop_Type_Tobacco,Crop_Type_Wheat
0,26,52,38,37,0,0,Urea,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,29,52,45,12,0,36,DAP,0,0,1,...,0,0,0,0,0,0,0,1,0,0
2,34,65,62,7,9,30,14-35-14,1,0,0,...,1,0,0,0,0,0,0,0,0,0
3,32,62,34,22,0,20,28-28,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,28,54,46,35,0,0,Urea,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [33]:
X = df.drop("Fertilizer",axis=1)
y = df["Fertilizer"]


In [35]:
# Splitting data into test and train
from sklearn.model_selection import train_test_split

X_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,shuffle=True)

import lightgbm as lgb

model = lgb.LGBMClassifier()
model.fit(X_train,y_train)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000066 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 94
[LightGBM] [Info] Number of data points in the train set: 79, number of used features: 6
[LightGBM] [Info] Start training from score -2.760010
[LightGBM] [Info] Start training from score -1.971553
[LightGBM] [Info] Start training from score -2.423538
[LightGBM] [Info] Start training from score -2.290006
[LightGBM] [Info] Start training from score -1.536235
[LightGBM] [Info] Start training from score -1.596859
[LightGBM] [Info] Start training from score -1.661398
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further

LGBMClassifier()

In [41]:
import numpy as np
def get_input(x):
    x_structure = {
        "Temparature": 0, "Humidity": 1, "Moisture": 2, "Nitrogen": 3,
        "Potassium": 4, "Phosphorous": 5, "Black": 6,  "Clayey": 7, "Loamy": 8,
        "Red": 9, "Sandy": 10, "Barley": 11, "Cotton": 12, "Ground Nuts": 13, "Maize": 14,
        "Millets": 15, "Oil seeds": 16, "Paddy": 17, "Pulses": 18, "Sugarcane": 19, "Tobacco": 20,
        "Wheat": 21
    }

    output = np.zeros(len(x_structure))
    output[0] = x[0]
    output[1] = x[1]
    output[2] = x[2]
    output[3] = x[3]
    output[4] = x[4]
    output[5] = x[5]
    output[x_structure[x[6]]] = 1
    output[x_structure[x[7]]] = 1
    return output

di = get_input([25,50,26,15,14,11,"Red","Ground Nuts"])

In [44]:
model.predict([di])[0]

'17-17-17'